In [ ]:
from sklearn import datasets, svm, metrics

original_Coviddataset_dir = '../CT_COVID_DATA/'

import matplotlib.pyplot as plt

In [ ]:
import os

train_covid_dir = "../train/covid/"
train_noncovid_dir = "../train/noncovid/"
validation_covid_dir = "../validation/covid/"
validation_noncovid_dir = "../validation/noncovid/"
test_covid_dir = "../test/covid/"
test_noncovid_dir = "../test/noncovid/"

print('total training cat images:', len(os.listdir(train_covid_dir)))
print('total training dog images:', len(os.listdir(train_noncovid_dir)))
print('total validation cat images:', len(os.listdir(validation_covid_dir)))
print('total validation dog images:', len(os.listdir(validation_noncovid_dir)))
print('total test cat images:', len(os.listdir(test_covid_dir)))
print('total test dog images:', len(os.listdir(test_noncovid_dir)))

In [ ]:
from keras.applications import VGG16

img_width, img_height = 224, 224

conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

In [ ]:
conv_base.summary()

In [ ]:
train_dir = "../train/"
validation_dir ="../validation/"
test_dir ="../test/"

import numpy as np

import os, shutil
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 7, 7, 512))
    labels = np.zeros(shape=(sample_count))

    generator = ImageDataGenerator(rescale=1./255).flow_from_directory(directory, 
    target_size=(img_width,img_height), batch_size = batch_size, class_mode='binary')

    i = 0

    print('Entering for loop...');

    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * 20 : (i + 1) * 20] = features_batch
        labels[i * 20 : (i + 1) * 20] = labels_batch
        i += 1
        print(i);
    
        if (i * 20) >= sample_count:
            break
    
    return features, labels

train_features, train_labels = extract_features(train_dir, 200)  
validation_features, validation_labels = extract_features(validation_dir, 100)
test_features, test_labels = extract_features(test_dir, 100)

In [ ]:
print(len(train_features[0]))

print(len(train_features[0][0]))

print(len(train_features[0][0][0]))

print(train_labels)

In [ ]:
svm_features = np.concatenate((train_features, validation_features))
svm_labels = np.concatenate((train_labels, validation_labels))

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve,GridSearchCV
from sklearn.svm import LinearSVC

print(svm_features.shape)
print(svm_labels.shape)

X_train, y_train = svm_features.reshape(300,7*7*512), svm_labels

param = [{
          "C": [0.01, 0.1, 1, 10, 100]
         }]

print(X_train.shape)
print(y_train.shape)

svm = LinearSVC(penalty='l2', loss='squared_hinge')  # As in Tang (2013)
clf = GridSearchCV(svm, param, cv=10)
clf.fit(X_train, y_train)

In [ ]:
clf.best_params_clf=LinearSVC(C=0.01)
clf=LinearSVC(C=0.01)
print(len(X_train))

In [ ]:
# Plot learning curves
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt

train_sizes, train_scores, test_scores = learning_curve(estimator = clf,
                                                        X = X_train,
                                                        y = y_train,
                                                        train_sizes = np.linspace(0.1,1.0,10),
                                                        cv = 10)
print(train_sizes)
print(len(train_scores))
print(len(test_scores))
print(train_scores)
print(test_scores)

train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

print(train_mean)
print(test_mean)

plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5,
         label='Training accuracy')

plt.fill_between(train_sizes,
                 train_mean + train_std,
                 train_mean - train_std,
                 alpha=0.15, color='blue')


plt.plot(train_sizes, test_mean, color='green', linestyle='--',
         marker='s', markersize=5, label='Validation accuracy')
plt.fill_between(train_sizes,
                 test_mean + test_std,
                 test_mean - test_std,
                 alpha=0.15, color='green')
plt.grid()
plt.xlabel('Number of training samples')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.ylim([0.5, 1.1])
plt.show()

In [ ]:
svm_features = np.concatenate((train_features, validation_features,test_features))
svm_labels = np.concatenate((train_labels, validation_labels,test_labels))
print(svm_features.shape)
print(svm_labels.shape)
X_train, y_train = svm_features.reshape(400,7*7*512), svm_labels
print(X_train.shape)
print(y_train.shape)

In [ ]:
# Evaluate model
from sklearn.model_selection import cross_val_score
print("\nAccuracy score (mean) :")
acc=cross_val_score(clf, X_train, y_train, cv=10)
print(acc)
print(np.mean(acc))
print("\nAccuracy score (standard deviation):")
print(np.std(acc))

In [ ]:
print(len(train_mean))
print(len(test_mean))
print(train_mean)
print(test_mean)